In [1]:
import transformers
import spacy
import pandas as pd
import json
import csv

In [2]:
import math

In [7]:
nlp = spacy.load("en_core_web_lg", disable=["tagger", "attribute_ruler", "lemmatizer"])

In [ ]:
bow_1 = nlp("black people colour skin african american race ")
bow_2 = nlp("black right people organisation human")

In [ ]:
bow_1.similarity(bow_2)

In [ ]:
# convert bows to str
# remove target term

In [ ]:
# importing search results
with open('/Users/anesterov/wd/jan31/wd_bows_en.json','r') as jf:
    wd_en = json.load(jf)
# importing related matches
with open('/Users/anesterov/reps/LODlit/bg/rm_bows_all.json','r') as jf:
    rm = json.load(jf)

In [ ]:
lang = "en"

In [ ]:
wd_df = pd.DataFrame(columns=['term', 'QID', 'wd_bow', 'spacy_cs'])

for term, hits in wd_en.items():
    
    if term in rm[lang].keys():
    
        # EN collecting related matches per term
        rm_per_term = []
        if rm[lang][term].get('aat'):
            rm_per_term.extend(rm[lang][term]['aat'])
        if rm[lang][term].get('wikidata'):
            rm_per_term.extend(rm[lang][term]['wikidata'])
        if rm[lang][term].get('pwn'):
            rm_per_term.extend(rm[lang][term]['pwn'])

        # remove target term from bow
        rm_per_term_no_target = list(set([t for t in rm_per_term if t != term]))
            
        # if there are search results
        if len(hits) > 0:
            for hit in hits:
                for i, bow in hit.items():
                    # remove target term from bow
                    wd_bow = list(set([t for t in bow if t != term]))
                    if len(wd_bow) > 0:
                        # calculate cs
                        cs = _get_sim(rm_per_term_no_target,wd_bow,nlp)
                        wd_df.loc[len(wd_df)] = [term,i,wd_bow,cs]
                    # if there are no tokens except the target term, cs == None
                    else:
                        wd_df.loc[len(wd_df)] = [term,i,wd_bow,None]
                    
    # if there are no related matches, cs == None, wd_bow == None
    else:
        wd_df.loc[len(wd_df)] = [term,i,None,None]

In [ ]:
# subfunction for spacy similarity
def _get_sim(bow_1:list, bow_2:list, nlp) -> float:
    '''
    bow_1 and bow_2: list, two bags of words; target term should be excluded
    Concerts lists to str of tokens separated by spaces
    Returns similarity score (float) rounded
    '''
    # converting list to str, spaces as a separator
    bow_1_str = ""
    for t in bow_1:
        bow_1_str += f"{t} "
        
    bow_2_str = ""
    for t in bow_2:
        bow_2_str += f"{t} "
        
    bow_1 = nlp(bow_1_str)
    bow_2 = nlp(bow_2_str)
        
    sim = bow_1.similarity(bow_2)
    
    return round(sim,3)

In [ ]:
wd_df.to_csv('spacy_sim_experiment.csv')

### Calculating AAT EN

In [6]:
def calculate_cs(bow_1:list, bow_2:list, nlp) -> float:
    '''
    Calculates cosine similarity between two bags of words;
    based on the spacy vectors
    bow_1 and bow_2: list, two bags of words;
    nlp: spacy nlp class loaded through spacy.load
    '''

    # converting list to str, spaces as a separator

    bow_1_str = ""
    for t in bow_1:
        bow_1_str += f"{t} "
        
    bow_2_str = ""
    for t in bow_2:
        bow_2_str += f"{t} "
        
    bow_1 = nlp(bow_1_str)
    bow_2 = nlp(bow_2_str)
        
    sim = round(bow_1.similarity(bow_2),3)
    
    return sim

In [3]:
# importing AAT with scopeNotes TF-IDF
with open('/Users/anesterov/reps/LODlit/AAT/aat_bows_tfidf_en.json','r') as jf:
    aat_tfidf_en = json.load(jf)

In [4]:
# importing background info
with open("/Users/anesterov/reps/LODlit/bg/rm_bows_tfidf_all.json","r") as jf:
    bg_info = json.load(jf)

In [5]:
lang = "en"

In [8]:
aat_df = pd.DataFrame(columns=['term', 'URI', 'aat_bow', 'cs_scopeNote_tfidf'])

for term, hits in aat_tfidf_en.items():
    
    if term in bg_info[lang].keys():
    
        # collecting bg info bow
        bg_combined = []
        if bg_info[lang][term].get('aat'):
            bg_combined.extend(bg_info[lang][term]['aat'])
        if bg_info[lang][term].get('wikidata'):
            bg_combined.extend(bg_info[lang][term]['wikidata'])
        if bg_info[lang][term].get('pwn'):
            bg_combined.extend(bg_info[lang][term]['pwn'])
        if bg_info[lang][term].get('wm'):
            bg_combined.extend(bg_info[lang][term]['wm'])

        # making a set
        bg_set = list(set(bg_combined))
            
        # if there are search results
        if len(hits) > 0:
            for hit in hits:
                for i, bow in hit.items():
                    # making a set
                    aat_bow = list(set(bow))
                    if len(aat_bow) > 0:
                        
                        # calculate cs
                        cs_combined = calculate_cs(bg_set,aat_bow,nlp)
                        
                        aat_df.loc[len(aat_df)] = [term,i,aat_bow,cs_combined]
                    
                    # if there are no tokens, cs == None
                    else:
                        aat_df.loc[len(aat_df)] = [term,i,aat_bow,None]
                    
    # if there are no related matches, cs == None
    else:
        aat_df.loc[len(aat_df)] = [term,i,aat_bow,None]

In [9]:
aat_df.to_csv('aat_cs_tfidf_en.csv')

### Calculating PWN CS

In [ ]:
# importing AAT
with open('/Users/anesterov/reps/LODlit/PWN/pwn31_bows.json','r') as jf:
    pwn = json.load(jf)

In [ ]:
pwn_df = pd.DataFrame(columns=['term', 'synset_ID', 'pwn_bow', 'spacy_cs'])

for term, hits in pwn.items():
    
    if term in rm[lang].keys():
    
        # EN collecting related matches per term
        rm_per_term = []
        if rm[lang][term].get('aat'):
            rm_per_term.extend(rm[lang][term]['aat'])
        if rm[lang][term].get('wikidata'):
            rm_per_term.extend(rm[lang][term]['wikidata'])
        if rm[lang][term].get('pwn'):
            rm_per_term.extend(rm[lang][term]['pwn'])

        # remove target term from bow
        rm_per_term_no_target = list(set([t for t in rm_per_term if t != term]))
        
        rm_string = ""
        for t in rm_per_term_no_target:
            rm_string += f"{t} "
            
        rm_bow_nlp = nlp(rm_string)
            
        # if there are search results
        if len(hits) > 0:
            for hit in hits:
                for i, bow in hit.items():
                    # remove target term from bow
                    aat_bow = list(set([t for t in bow if t != term]))
                    if len(aat_bow) > 0:
                        aat_str = ""
                        for t in aat_bow:
                            aat_str += f"{t} "
                        aat_bow_nlp = nlp(aat_str)
                        
                        # calculate cs
                        cs = round(rm_bow_nlp.similarity(aat_bow_nlp),3)
                        aat_df.loc[len(aat_df)] = [term,i,aat_bow,cs]
                    # if there are no tokens except the target term, cs == None
                    else:
                        aat_df.loc[len(aat_df)] = [term,i,aat_bow,None]
                    
    # if there are no related matches, cs == None
    else:
        aat_df.loc[len(aat_df)] = [term,i,aat_bow,None]